In [35]:
import os
import cv2
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import ipywidgets as widgets

In [36]:
# Load CSV and video path
csv_path = r"C:\\Users\\sanke\\Desktop\\Therapist_Model\\Segmentation Data\\Data\\Final Data\\Final_Conversation.csv"
video_path = r"C:\\Users\\sanke\\Desktop\\Therapist_Model\\Segmentation Data\\Data\\Videos\\Telemental Health Mock Session.mp4"
final_conversation_df = pd.read_csv(csv_path)

In [37]:
# Filter only client timestamps
client_df = final_conversation_df[final_conversation_df['Speaker'].str.lower() == 'client'].copy()
client_df.head()

,Start,End,Speaker,Text
1,01:16,01:27,Client,"And I'm a peer educator at CAPS here, and I t..."
3,01:53,02:08,Client,"Um, so nothing much has changed in terms of w..."
4,02:08,02:21,Client,But as for the homework. I felt that sometime...
5,02:22,02:33,Client,and my thoughts were like controlling me. So ...
7,04:41,04:55,Client,So whenever I felt like my anxiety was throug...


In [38]:
# Convert MM:SS to seconds
def time_to_seconds(t):
    minutes, seconds = map(int, t.split(':'))
    return minutes * 60 + seconds
client_df['Start_sec'] = client_df['Start'].apply(time_to_seconds)
client_df['End_sec'] = client_df['End'].apply(time_to_seconds)
client_df['Mid_sec'] = (client_df['Start_sec'] + client_df['End_sec']) / 2

In [39]:
# Face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [40]:
# Output directory
output_dir = r"C:\\Users\\sanke\\Desktop\\Therapist_Model\\Extracted_Faces"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Dropdown widgets
view_dropdown = widgets.Dropdown(
    options=['', 'Gallery View', 'Speaker View'],
    description='Video View:',
    style={'description_width': 'initial'}
)
option_dropdown = widgets.Dropdown(
    options=[''],
    description='Face Option:',
    style={'description_width': 'initial'}
)
confirm_button = widgets.Button(description="Confirm Selection", button_style='success')
output_area = widgets.Output()

def update_option_dropdown(change):
    if change['new'] == 'Gallery View':
        option_dropdown.options = ['', 'Left', 'Right']
    elif change['new'] == 'Speaker View':
        option_dropdown.options = ['', 'Large', 'Small']
    else:
        option_dropdown.options = ['']
view_dropdown.observe(update_option_dropdown, names='value')

def confirm_selection(b):
    selected_view['value'] = view_dropdown.value
    selected_option['value'] = option_dropdown.value
    with output_area:
        clear_output()
        if selected_view['value'] and selected_option['value']:
            print(f"Selected View: {selected_view['value']} | Selected Option: {selected_option['value']}")
        else:
            print("Please make selections in both dropdowns.")

confirm_button.on_click(confirm_selection)

display(view_dropdown, option_dropdown, confirm_button, output_area)

Dropdown(description='Video View:', options=('', 'Gallery View', 'Speaker View'), style=DescriptionStyle(descr…

Dropdown(description='Face Option:', options=('',), style=DescriptionStyle(description_width='initial'), value…

Button(button_style='success', description='Confirm Selection', style=ButtonStyle())

Output()

In [42]:
# Face extraction function
def extract_face_from_frame(frame, view_type, option, idx, target_size=(256, 256)):
    h, w, _ = frame.shape

    def save_resized_face(face, prefix):
        resized_face = cv2.resize(face, target_size, interpolation=cv2.INTER_AREA)
        path = os.path.join(output_dir, f"{prefix}_{idx}.jpg")
        cv2.imwrite(path, resized_face)
        return path

    if view_type == 'Gallery View':
        if option == 'Left':
            half_img = frame[:, :w//2]
            faces = face_cascade.detectMultiScale(half_img, 1.3, 5)
            if len(faces) == 1:
                x, y, fw, fh = faces[0]
                face = half_img[y:y+fh, x:x+fw]
                return save_resized_face(face, "left")
        elif option == 'Right':
            half_img = frame[:, w//2:]
            faces = face_cascade.detectMultiScale(half_img, 1.3, 5)
            if len(faces) == 1:
                x, y, fw, fh = faces[0]
                face = half_img[y:y+fh, x:x+fw]
                return save_resized_face(face, "right")

    elif view_type == 'Speaker View':
        faces = face_cascade.detectMultiScale(frame, 1.3, 5)
        if len(faces) == 2:
            face_areas = [(fw * fh, (x, y, fw, fh)) for (x, y, fw, fh) in faces]
            face_areas.sort(reverse=True)
            if option == 'Large':
                (xL, yL, fwL, fhL) = face_areas[0][1]
                face = frame[yL:yL+fhL, xL:xL+fwL]
                return save_resized_face(face, "large")
            elif option == 'Small':
                (xS, yS, fwS, fhS) = face_areas[1][1]
                face = frame[yS:yS+fhS, xS:xS+fwS]
                return save_resized_face(face, "small")

    return None

In [43]:
# Processing code to be run after selection
if selected_view['value'] is None or selected_option['value'] is None:
    print("Please select both dropdown values before processing.")
else:
    print(f"Processing with: {selected_view['value']} - {selected_option['value']}")
    cap = cv2.VideoCapture(video_path)

    for idx, row in client_df.iterrows():
        cap.set(cv2.CAP_PROP_POS_MSEC, row['Mid_sec'] * 1000)
        ret, frame = cap.read()
        if not ret:
            continue

        path = extract_face_from_frame(frame, selected_view['value'], selected_option['value'], idx)
        if path:
            client_df.loc[idx, 'Image_Path'] = path

    cap.release()

    if 'Image_Path' not in final_conversation_df.columns:
        final_conversation_df['Image_Path'] = None
    for idx in client_df.index:
        final_conversation_df.at[idx, 'Image_Path'] = client_df.at[idx, 'Image_Path']

    post_extraction_df = final_conversation_df.copy()
    print("Done! Showing result:")
    display(post_extraction_df[['Start', 'End', 'Speaker', 'Text', 'Image_Path']].head())

Processing with: Gallery View - Left
Done! Showing result:


,Start,End,Speaker,Text,Image_Path
0,00:01,01:16,therapist,"All right, hi everybody. My name is Daniel Ga...",None
1,01:16,01:27,Client,"And I'm a peer educator at CAPS here, and I t...",C:\\Users\\sanke\\Desktop\\Therapist_Model\\Ex...
2,01:27,01:53,therapist,"All right, so we're going to go ahead and div...",None
3,01:53,02:08,Client,"Um, so nothing much has changed in terms of w...",C:\\Users\\sanke\\Desktop\\Therapist_Model\\Ex...
4,02:08,02:21,Client,But as for the homework. I felt that sometime...,C:\\Users\\sanke\\Desktop\\Therapist_Model\\Ex...
